# Prediction


In [10]:
import pickle
import os
import numpy as np
import pandas as pd
import sys
sys.path.append('qsar')
from Prediction import predict

data_path = os.getcwd() + '/Data/repurpose_pains.csv'
SAVE_PREFIX  = os.getcwd() + '/utility/'
activity_col = 'pChEMBL Value'
task_type = 'C'
target_thresh = 7
var_thresh = 0.05
scoring = 'f1'

In [11]:
data = pd.read_csv(data_path)
data.head()

,Unnamed: 0,Name,CanonSmiles,logp,mw,tpsa,n_hba,n_hbd,ro5,pfizer_rule_passed,...,pains_MatchedNames,alarmnmr_Disposed,alarmnmr_MatchedAtoms,alarmnmr_MatchedNames,bms_Disposed,bms_MatchedAtoms,bms_MatchedNames,chelator_Disposed,chelator_MatchedAtoms,chelator_MatchedNames
0,21,Pyridoxal phosphate,Cc1ncc(COP(=O)(O)O)c(C=O)c1O.O,-0.30728,265.035138,148.45,5,3,4,True,...,['-'],Rejected,"[((15, 14, 11, 4, 3, 2, 1),), ((11, 12, 13),)]","['c1ccccc1O', '[CH]=O']",Rejected,"[((11, 12, 13),)]",['aldehyde'],Accepted,['-'],['-']
1,24,Histidine,N[C@@H](Cc1c[nH]cn1)C(=O)O,-0.63590,155.069477,92.00,3,3,5,True,...,['-'],Accepted,['-'],['-'],Accepted,['-'],['-'],Accepted,['-'],['-']
2,26,Pyruvic acid,CC(=O)C(=O)O,-0.34000,88.016044,54.37,2,1,5,True,...,['-'],Accepted,['-'],['-'],Rejected,"[((1, 2, 3, 4),)]",['alpha_dicarbonyl'],Accepted,['-'],['-']
3,27,Phenylalanine,N[C@@H](Cc1ccccc1)C(=O)O,0.64100,165.078979,63.32,2,2,5,True,...,['-'],Accepted,['-'],['-'],Accepted,['-'],['-'],Accepted,['-'],['-']
4,28,Biotin,O=C(O)CCCCC1SCC2NC(=O)NC21,0.79680,244.088163,78.43,3,3,5,True,...,['-'],Rejected,"[((7, 8, 9),)]",['CSC'],Accepted,['-'],['-'],Accepted,['-'],['-']


In [42]:
data.shape

(8333, 3)

In [13]:
data = data[['Name','CanonSmiles']]
data.rename(columns = {'Name':'ID','CanonSmiles':'SMILES'}, inplace = True)
data.head(2)

/tmp/ipykernel_1073869/496460073.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = {'Name':'ID','CanonSmiles':'SMILES'}, inplace = True)


,ID,SMILES
0,Pyridoxal phosphate,Cc1ncc(COP(=O)(O)O)c(C=O)c1O.O
1,Histidine,N[C@@H](Cc1c[nH]cn1)C(=O)O


In [14]:
from rdkit import Chem
from rdkit import Chem, DataStructs
data['Molecule'] = data['SMILES'].apply(Chem.MolFromSmiles)
def RDKFp(mol, maxPath=7, fpSize=4096, nBitsPerHash=2):
        fp = Chem.RDKFingerprint(mol, maxPath=maxPath, fpSize=fpSize, nBitsPerHash=nBitsPerHash)
        ar = np.zeros((1,), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(fp, ar)
        return ar
RDKF = data.copy()
RDKF["FPs"] = RDKF.Molecule.apply(RDKFp)
X = np.stack(RDKF.FPs.values)
df = pd.DataFrame(X)
RDKF_visualize= pd.concat([RDKF, df], axis = 1).drop([RDKF.columns[1],"FPs", "Molecule"], axis =1)
RDKF_visualize['pChEMBL Value'] = 0
RDKF_visualize.columns = RDKF_visualize.columns.astype('string')

[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not removing hydrogen atom without neighbors
[13:42:15] WARNING: not r

In [15]:
pred = predict(materials_path =SAVE_PREFIX,
               data = RDKF_visualize,
               activity_col=activity_col,
              ID = 'ID')
Report = pred.predict()

The number of active structures: 44
Percentage of active structures: 0.5280211208448338


/home/labhhc/anaconda3/envs/QSAR/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [22]:
final = pd.concat([data, Report.drop(['Index'], axis =1)], axis=1)


In [23]:
final.sort_values(by='Probability', ascending = False).head()

,ID,SMILES,Molecule,Probability,Predict
6597,GSK-364735,Cn1c(=O)c(C(=O)NCCO)c(O)c2ncc(Cc3ccc(F)cc3)cc21,<rdkit.Chem.rdchem.Mol object at 0x7f682ec40f90>,99.592064,1
5876,Elvitegravir,COc1cc2c(cc1Cc1cccc(Cl)c1F)c(=O)c(C(=O)O)cn2[C...,<rdkit.Chem.rdchem.Mol object at 0x7f682ecacf20>,98.818932,1
5756,Dolutegravir,C[C@@H]1CCO[C@H]2Cn3cc(C(=O)NCc4ccc(F)cc4F)c(=...,<rdkit.Chem.rdchem.Mol object at 0x7f682eca9a50>,98.627785,1
8227,Cabotegravir,C[C@H]1CO[C@@H]2Cn3cc(C(=O)NCc4ccc(F)cc4F)c(=O...,<rdkit.Chem.rdchem.Mol object at 0x7f682ec6e430>,98.067894,1
5411,3-({4-[(6-CHLORO-1-BENZOTHIEN-2-YL)SULFONYL]-2...,N=C(N)c1cccc(CN2CCN(S(=O)(=O)c3cc4ccc(Cl)cc4s3...,<rdkit.Chem.rdchem.Mol object at 0x7f682eca00b0>,81.362564,1


In [41]:
final[final['Predict']>0].to_csv('final_screen.csv')